In [ ]:
from pprint import *
from ib_insync import *
util.startLoop()  # uncomment this line when in a notebook

ib = IB() # making a IB class
ib.connect('127.0.0.1', 7497, clientId=4)

In [ ]:
f = Future(symbol='MES', lastTradeDateOrContractMonth='202403', exchange='CME', currency='USD') # valid instrument - Future
contract = ib.qualifyContracts(f)[0]
pprint(contract)

In [ ]:
# first, get a market order

mkt_ord = MarketOrder('BUY', totalQuantity=10)
mkt_trades = ib.placeOrder(contract, mkt_ord)

# print('#' * 100)
ib.sleep(2)
# pprint(mkt_trades) #等2秒过后，这个时候的order已经被fill了，所以打印出来的信息，就已经包括了所有的order的信息了

print('#' * 100)
print('Fill price is: ' + str(mkt_trades.fills[0].execution.price))


# second, get a STOP sell order

stp_ord = StopOrder(action='SELL', totalQuantity=10, stopPrice=mkt_trades.fills[0].execution.price - 10, tif='GTC', outsideRth=True)
stp_trades = ib.placeOrder(contract, stp_ord)

ib.sleep(2)
# pprint(stp_trades) #等2秒过后，看看这个order的stop lost price对不对


# third get a profit taken limit order for 75-80% of the quantity

lmt_ord = LimitOrder(action='SELL', totalQuantity=8, lmtPrice=mkt_trades.fills[0].execution.price + 10)
lmt_trades = ib.placeOrder(contract, lmt_ord)

ib.sleep(2)
# pprint(trades) #等2秒过后，看看这个order的profit taken price对不对



# 查看一下现在active的order

open_ord = ib.reqOpenOrders()

# pprint(open_ord)

for ord in open_ord:
    # print(ord.order)
    if ord.order.orderType == 'STP':
        existing_stp_ord = ord
        print('Stop loss order -----> ' + str(ord.order.totalQuantity) + ' at the price: ' + str(ord.order.auxPrice))
        # print(ord.order.orderId)
        stp_ord_id = ord.order.orderId
        # print(ord.orderStatus)
        stp_ord_status = ord.orderStatus.status
        # print('#' * 100)
    if ord.order.orderType == 'LMT':
        existing_lmt_ord = ord
        print('Profit taken order -----> ' + str(ord.order.totalQuantity) + 'at the price: ' + str(ord.order.lmtPrice))
        # print(ord.order.orderId)
        lmt_ord_id = ord.order.orderId
        # print(ord.orderStatus)
        lmt_ord_status = ord.orderStatus.status
        # print('#' * 100)

print('&&' * 100)
pprint(existing_stp_ord)
print('#' * 100)
pprint(existing_lmt_ord)
print('#' * 100)
print('STP Order ----->' + ' Order ID: ' + str(stp_ord_id) + '     Order Status: ' + stp_ord_status)
print('LMT Order ----->' + ' Order ID: ' + str(lmt_ord_id) + '     Order Status: ' + lmt_ord_status )
